# Finding baselines to beat with model
Used to predict next x months by just repeating observered volume.  
Measuring the model's effectiveness.
If the model can't beat these it's practically useless

https://towardsdatascience.com/how-to-build-a-baseline-model-be6ce42389fc

In [19]:
import numpy as np
import pandas as pd

from pytorch_forecasting import Baseline, TemporalFusionTransformer, TimeSeriesDataSet
from torch.utils.data import DataLoader
from torch.utils.data import Dataset

from pytorch_forecasting.data import GroupNormalizer

from ipynb.fs.full.training_preprocessing import GetDataset

In [20]:
from sklearn.metrics import mean_absolute_error, mean_squared_error

In [21]:
df = GetDataset()
df

,Timestamp,subtotal,transaction_count,rain,temperature,holiday,oil_price,workforce_type_1,workforce_type_2,workforce_type_3,workforce_type_4,time_idx,constant_group,year,month,day,hour
0,2021-05-17 10:00:00,428.03,11.0,0.0,12.7,0,69.62,1.0,2.0,3.0,0.0,0,0,2021,5,17,10
1,2021-05-17 11:00:00,324.78,15.0,0.0,13.4,0,69.62,1.0,2.0,3.0,1.0,1,0,2021,5,17,11
2,2021-05-17 12:00:00,279.75,12.0,0.0,13.6,0,69.62,1.0,2.0,3.0,1.0,2,0,2021,5,17,12
3,2021-05-17 13:00:00,1630.59,29.0,0.0,14.1,0,69.62,1.0,2.0,3.0,1.0,3,0,2021,5,17,13
4,2021-05-17 14:00:00,1166.10,18.0,0.0,15.9,0,69.62,1.0,2.0,3.0,1.0,4,0,2021,5,17,14
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5062,2022-11-30 14:00:00,4779.11,82.0,0.0,0.0,0,83.45,1.0,4.0,5.0,0.0,5062,0,2022,11,30,14
5063,2022-11-30 15:00:00,2460.84,54.0,0.0,0.0,0,83.45,1.0,4.0,5.0,0.0,5063,0,2022,11,30,15
5064,2022-11-30 16:00:00,2026.30,43.0,0.0,0.0,0,83.45,1.0,4.0,5.0,0.0,5064,0,2022,11,30,16
5065,2022-11-30 17:00:00,2346.13,44.0,0.0,0.0,0,83.45,1.0,4.0,5.0,0.0,5065,0,2022,11,30,17


In [22]:
# Given range, see how close the mean of previous 2 weeks and next 2 weeks is to the actual value
# Moving average

date_to_predict = pd.to_datetime('2022-10-20 10:00:00')


df['Timestamp'] = pd.to_datetime(df['Timestamp'])

# Get the transaction_count values from the last week of df
prev_week_1 = df[(df['Timestamp'] >= date_to_predict - pd.Timedelta(days=7)) & (df['Timestamp'] < date_to_predict)]['transaction_count']
prev_week_2 = df[(df['Timestamp'] >= date_to_predict - pd.Timedelta(days=15)) & (df['Timestamp'] < date_to_predict - pd.Timedelta(days=7))]['transaction_count']

next_week_1 = df[(df['Timestamp'] <= date_to_predict + pd.Timedelta(days=14)) & (df['Timestamp'] > date_to_predict + pd.Timedelta(days=7))]['transaction_count']
next_week_2 = df[(df['Timestamp'] <= date_to_predict + pd.Timedelta(days=21)) & (df['Timestamp'] > date_to_predict + pd.Timedelta(days=14))]['transaction_count']

actual_week = df[(df['Timestamp'] >= date_to_predict) & (df['Timestamp'] < date_to_predict + pd.Timedelta(days=7))]['transaction_count']

# For each field in last_2_weeks and next_2_weeks, get the mean between them 
# and store it in a new dataframe
predicted_with_means = []
for i in range (len(prev_week_1)):
    # Append to df2
    predicted_with_means.append((prev_week_1.iloc[i] + prev_week_2.iloc[i] + next_week_1.iloc[i] + next_week_2.iloc[i]) / 4)


# Get the mean absolute error between the actual values and the predicted values
mae = mean_absolute_error(actual_week, predicted_with_means)
print(f'Mean Absolute Error: {mae}')

Mean Absolute Error: 8.698412698412698


In [23]:
date_to_predict = pd.to_datetime('2022-10-20 10:00:00')

prev_date_1 = df[df['Timestamp'] == date_to_predict - pd.Timedelta(weeks=1)]['transaction_count'].values[0]
prev_date_2 = df[df['Timestamp'] == date_to_predict - pd.Timedelta(weeks=2)]['transaction_count'].values[0]

next_date_1 = df[df['Timestamp'] == date_to_predict + pd.Timedelta(weeks=1)]['transaction_count'].values[0]
next_date_2 = df[df['Timestamp'] == date_to_predict + pd.Timedelta(weeks=2)]['transaction_count'].values[0]

predicted = (prev_date_1 + prev_date_2 + next_date_1 + next_date_2) / 4

actual = df[df['Timestamp'] == date_to_predict]['transaction_count'].values[0]

print(f'Predicted: {predicted}, Actual: {actual}')
print(f'Difference: {abs(predicted - actual)}')

Predicted: 13.25, Actual: 11.0
Difference: 2.25


In [24]:
# Calculate the Mean Squared Error
mse = mean_squared_error(actual_week, predicted_with_means)

print(f'Mean Squared Error: {mse}')

Mean Squared Error: 137.77579365079364


In [28]:
import torch
actual_week_ar = np.array(actual_week)

dist = torch.dist(torch.FloatTensor(actual_week_ar), torch.FloatTensor(predicted_with_means))
dist

tensor(93.1658)

In [29]:
# Calculate SMAPE between actual and predicted values.
def smape(actual, predicted):
    actual = np.array(actual)
    predicted = np.array(predicted)
    smape_val = (100.0 / actual.size) * np.sum(2.0 * np.abs(predicted - actual) / (np.abs(actual) + np.abs(predicted)))
    return smape_val

In [30]:
smape = smape(actual_week[:63], predicted_with_means[:63])
print(smape)

33.96301799758602
